In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras

from src.config import c as gc
from src.generator import Generator

In [2]:
BATCH = 64
IMAGE_SIZE = (224, 224)

In [3]:
gen = Generator(
    csv_file=f"{gc['DATA_DIR']}/src/train.csv",
    images_src_dir=f"{gc['DATA_DIR']}/src/train_images",
    batch_size=BATCH,
    target_image_size=IMAGE_SIZE,
    #cache_dir=None,
    cache_dir=gc["DATA_DIR"] + "/images_cache",
    shuffle=True,
)

In [4]:
x0, y0 = gen.get_one(0)

In [5]:
base_model = keras.applications.ResNet50(
    weights="imagenet", include_top=False, input_shape=x0.shape, classes=y0.shape[0]
)

x = base_model.output
x = keras.layers.GlobalAveragePooling2D(name="avg_pool")(x)
x = keras.layers.Flatten(name="flatten")(x)
x = keras.layers.Dense(y0.shape[0], activation="sigmoid")(x)

model = keras.Model(inputs=base_model.inputs, outputs=[x])

In [6]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(
    x=gen,
    epochs=10,
    verbose=1,
    max_queue_size=10,
    workers=20,
    use_multiprocessing=True,
)

Epoch 1/10
